In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
# Cargar datasets
df_orders = pd.read_excel('./datasets/df_orders.xlsx')
df_vehicle = pd.read_excel('./datasets/df_vehicle.xlsx')
df_distance_km = pd.read_excel('./datasets/df_distance_km.xlsx')

In [3]:
# Preprocesar datos
df_orders[["mes", "año"]] = df_orders["mes_anio"].str.split("-", expand=True).astype(int)
df_orders.drop(columns=["mes_anio"], inplace=True)
df_orders["cliente"] = df_orders["cliente"].apply(lambda x: f'cliente_{x}')

In [4]:


# La matriz de distancia tiene columnas de clientes y almacén pero no filas, le ponemos las columnas en las filas para tener ok la matriz de distancia
clientes_y_almacen = [f"cliente_{i+1}" for i in range(len(df_orders))] + ["Almacén"]
df_distance_km.index = clientes_y_almacen
df_distance_km.columns = clientes_y_almacen

df_orders["cliente"] = [f"cliente_{i+1}" for i in range(len(df_orders))]

# Convertir la matriz de distancias a un diccionario
dicc_distancia = df_distance_km.to_dict()

# Crear el problema de optimización
prob = pulp.LpProblem("Optimización_Rutas_P1", pulp.LpMinimize)

# Parámetros
clientes = [f"cliente_{i+1}" for i in range(len(df_orders))]
vehiculos = list(df_vehicle["vehiculo_id"])
demanda = dict(zip(df_orders["cliente"], df_orders["order_demand"]))
capacidad_vehiculo = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["capacidad_kg"]))
coste_km = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["costo_km"]))
autonomia = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["autonomia_km"]))
num_viajes = 3  # Máximo número de viajes por vehículo
nombre_almacen = "Almacén"

# Variables de decisión: 1 si el vehículo v atiende al cliente i en el viaje t, 0 si no
x = pulp.LpVariable.dicts(
    "x", [(i, v, t) for i in clientes for v in vehiculos for t in range(num_viajes)], cat="Binary"
)
y = pulp.LpVariable.dicts(
    "y", [(v, t) for v in vehiculos for t in range(num_viajes)], cat="Binary"
)

# Restricción 1: Cada cliente debe ser atendido por suficiente capacidad
for i in clientes:
    prob += pulp.lpSum(x[(i, v, t)] * capacidad_vehiculo[v] for v in vehiculos for t in range(num_viajes)) >= demanda[i], f"Satisfacer_demanda_{i}"

# Restricción 2: La capacidad total de un vehículo en un viaje no puede exceder su capacidad
for v in vehiculos:
    for t in range(num_viajes):
        prob += pulp.lpSum(demanda[i] * x[(i, v, t)] for i in clientes) <= capacidad_vehiculo[v], f"Capacidad_vehiculo_{v}_viaje_{t}"

# Restricción 3: La autonomía del vehículo no puede ser excedida en cada viaje
for v in vehiculos:
    for t in range(num_viajes):
        prob += pulp.lpSum(
            (dicc_distancia[nombre_almacen][i] + dicc_distancia[i][nombre_almacen]) * x[(i, v, t)]
            for i in clientes
        ) <= autonomia[v], f"Autonomía_vehiculo_{v}_viaje_{t}"

# Restricción 4: Evitar rutas con distancia 0
for i in clientes:
    for j in clientes + [nombre_almacen]:
        if i != j and dicc_distancia[i][j] == 0:
            for v in vehiculos:
                for t in range(num_viajes):
                    prob += x[(i, v, t)] + x[(j, v, t)] <= 1, f"Ruta_no_valida_{i}_a_{j}_vehiculo_{v}_viaje_{t}"

# Función objetivo: Minimizar el coste total considerando todos los viajes
prob += pulp.lpSum(
    x[(i, v, t)] * (dicc_distancia[nombre_almacen][i] + dicc_distancia[i][nombre_almacen]) * coste_km[v]
    for i in clientes for v in vehiculos for t in range(num_viajes)
), "Minimizar_coste"

# Resolver el problema
prob.solve()

# Mostrar resultados
print(f"Estado del problema: {pulp.LpStatus[prob.status]}")

if pulp.LpStatus[prob.status] == "Optimal":
    solucion = {
        (i, v, t): pulp.value(x[(i, v, t)]) for i in clientes for v in vehiculos for t in range(num_viajes) if pulp.value(x[(i, v, t)]) > 0
    }
    rutas_por_vehiculo = {v: {"viajes": [{} for _ in range(num_viajes)]} for v in vehiculos}

    for (i, v, t), valor in solucion.items():
        if valor > 0:
            viaje = rutas_por_vehiculo[v]["viajes"][t]
            if "ruta" not in viaje:
                viaje["ruta"] = []
                viaje["carga"] = 0
                viaje["distancia"] = 0
                viaje["coste"] = 0
            viaje["ruta"].append(i)
            viaje["carga"] += demanda[i]

    for v, datos in rutas_por_vehiculo.items():
        for t, viaje in enumerate(datos["viajes"]):
            if "ruta" in viaje and viaje["ruta"]:
                ruta = [nombre_almacen] + viaje["ruta"] + [nombre_almacen]
                distancia_total = 0
                for j in range(len(ruta) - 1):
                    distancia_total += dicc_distancia[ruta[j]][ruta[j + 1]]
                viaje["distancia"] = distancia_total
                viaje["coste"] = distancia_total * coste_km[v]

    print("\nDetalles por vehículo y viajes:")
    for v, datos in rutas_por_vehiculo.items():
        for t, viaje in enumerate(datos["viajes"]):
            if "ruta" in viaje and viaje["ruta"]:
                print(f"Vehículo {v}, Viaje {t + 1}:")
                print(f"  - Ruta: Almacén -> {' -> '.join(viaje['ruta'])} -> Almacén")
                print(f"  - Carga total de la ruta: {viaje['carga']} kg")
                print(f"  - Distancia total de la ruta: {viaje['distancia']:.2f} km")
                print(f"  - Coste total de la ruta: {viaje['coste']:.2f} €")
                print("-" * 60)
    
    coste_total = pulp.value(prob.objective)
    print(f"\nCoste total de toda la entrega de suministro: {coste_total:.2f} €")
else:
    print("No se encontró una solución óptima para las peticiones.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/pedrohd/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/6s/rbb4xdjd1d15rp4z5_w0yt8c0000gn/T/63ed191dfaf04857978abfec734a8306-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/6s/rbb4xdjd1d15rp4z5_w0yt8c0000gn/T/63ed191dfaf04857978abfec734a8306-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 565 COLUMNS
At line 3734 RHS
At line 4295 BOUNDS
At line 4656 ENDATA
Problem MODEL has 560 rows, 360 columns and 2088 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10.8529 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 54 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 54 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 54 strengthened rows, 0 substitutions
Cgl0004I processe